In [ ]:
print('pk')

In [2]:
#imports:
# !pip install
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from datagenerator import *
print(tf.version.VERSION)
print('ok')

2.4.1
ok


In [3]:
# constants:
SIZE = 5 # todo: change from 170
RESIZED_WIDTH = SIZE  # temp
RESIZED_HEIGHT = SIZE  # temp
RESIZED_NUM_PIXELS = RESIZED_WIDTH * RESIZED_HEIGHT
SHAPE = RESIZED_NUM_PIXELS + 3
TRAIN_PATH = r'R:\domain_lab\runs_processed\train'  # temp
VALID_PATH = r'R:\domain_lab\runs_processed\validation'  # temp
TEST_PATH = r'R:\domain_lab\runs_processed\test'  # temp
BACH_SIZE = 3 # temp
TRAIN_DATASET_SIZE = dataset_size(TRAIN_PATH)
VALID_DATASET_SIZE = dataset_size(VALID_PATH)
TEST_DATA_SIZE = dataset_size(TEST_PATH)
STEPS_PER_EPOCH = TRAIN_DATASET_SIZE // BACH_SIZE
EPOCHS = 2 # temp
VALIDATION_STEPS = VALID_DATASET_SIZE // BACH_SIZE
TEST_STEPS = TEST_DATA_SIZE // BACH_SIZE
IMAGE_SHAPE = (RESIZED_WIDTH, RESIZED_HEIGHT, 1)

print('ok')

ok


In [4]:
# from pprint import pprint
# import PIL as pl
# def data_generator(path):
#     i=0
#     images=[]
#     vol=[]
#     for root, dirs, files in os.walk(path):
#         for name in files:
#             vol.append(get_volt_from_img_name(name))
#     while i<=len(range(1,len(vol))):
#         try:
#             yield {'image':np.array(pl.Image.open(os.path.join(root,files[i-1]))), 'voltage_data':[vol[i],vol[i-1],vol[i]-vol[i-1]]}, np.array(pl.Image.open(os.path.join(root,files[i])))
#             i+=1
#         except StopIteration:
#             break

In [5]:
# activating data gen:
# x_train_generator = tf.data.Dataset.from_generator(data_generator)

foo = data_generator_for_cnn
train_generator = foo(TRAIN_PATH, BACH_SIZE, True, RESIZED_WIDTH, RESIZED_HEIGHT)
validation_generator = foo(VALID_PATH, BACH_SIZE, True, RESIZED_WIDTH, RESIZED_HEIGHT)
test_generator = foo(TEST_PATH, BACH_SIZE, True, RESIZED_WIDTH, RESIZED_HEIGHT)

types = ( (tf.float32,tf.float32), (tf.float32) ) 
shapes = ((list(IMAGE_SHAPE),[3]),
          (list(IMAGE_SHAPE)))
train_data = tf.data.Dataset.from_generator(creat_next_data_for_cnn(TRAIN_PATH,  RESIZED_WIDTH, RESIZED_HEIGHT),
                                      output_types=types,
                                      output_shapes=shapes
                                     ).batch(BACH_SIZE).repeat()

valid_data = tf.data.Dataset.from_generator(creat_next_data_for_cnn(VALID_PATH,  RESIZED_WIDTH, RESIZED_HEIGHT),
                                      output_types=types,
                                      output_shapes=shapes
                                     ).batch(BACH_SIZE).repeat()

print('ok')

ok


In [6]:
import tensorflow.keras as K
import tensorflow.keras.layers as L

image=L.Input(shape=IMAGE_SHAPE, name = 'image')
voltage_data=L.Input(shape=(3,), name = 'voltage_data')



# conv branch
conv = L.Conv2D(filters=1, kernel_size=1, activation="sigmoid")(image)
conv = L.Conv2D(filters=1, kernel_size=1, activation="sigmoid")(conv)
conv = L.Conv2D(filters=1, kernel_size=1, activation="sigmoid")(conv)
conv_flat = L.Flatten()(conv)
conv_out = L.Concatenate(axis=1)([conv_flat,voltage_data])
flat_hidden = L.Dense(units=128, activation="sigmoid")(conv_out)
flat_hidden = L.Dense(units=128, activation="sigmoid")(flat_hidden)
flat_hidden = L.Dense(units=128, activation="sigmoid")(flat_hidden)
flat_hidden = L.Dense(units=128, activation="sigmoid")(flat_hidden)
flat_hidden = L.Dense(units=128, activation="sigmoid")(flat_hidden)
flat_out = L.Dense(units=128, activation="sigmoid")(flat_hidden)
# comb
comb_out = L.Dense(units=RESIZED_NUM_PIXELS, activation="sigmoid")(flat_out)

# conv branch
model = K.Model(inputs=[image,voltage_data] , outputs=comb_out)
# model.summary()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
# model.fit(x=train_data,
#           steps_per_epoch=STEPS_PER_EPOCH,
#           epochs=EPOCHS,
#           validation_data=valid_data,
#           validation_steps=VALIDATION_STEPS)
my_fit(model, 
       train_generator, 
       validation_generator, 
       batch_size=BACH_SIZE, 
       epochs= EPOCHS)
# model.fit(x=train_generator, 
#           steps_per_epoch=STEPS_PER_EPOCH, 
#           epochs=EPOCHS, 
#           validation_data=validation_generator)
print('ok')

ha
1
ha
2
ha
3
all images:  (3, 5, 5, 1)
all size 3:  (3, 3)
image:  (5, 5, 1)
size3:  (3,)
all image (3, 5, 5, 1)
image (5, 5, 1)


ValueError: in user code:

    C:\Users\SMR_Admin\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\SMR_Admin\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\SMR_Admin\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\SMR_Admin\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\SMR_Admin\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\SMR_Admin\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\traini

In [ ]:
# Evaluating the Model:
test_loss = model.evaluate(test_generator, steps=TEST_STEPS)
print('Test loss:', test_loss)


In [ ]:
######## functions for visualizing predictions ########

def visualize_prediction(pred_input, original_output, prediction):
    fig, axes = plt.subplots(1, 3, figsize=(9, 3))

    # Plot the input frame.
    ax = axes[0]
    ax.imshow(np.squeeze(pred_input), cmap="gray")
    ax.set_title(f"input")
    ax.axis("off")

    # Plot the original new frame.
    ax = axes[1]
    ax.imshow(np.squeeze(original_output), cmap="gray")
    ax.set_title(f"original output")
    ax.axis("off")

    # Plot the new frame.
    ax = axes[2]
    ax.imshow(np.squeeze(prediction), cmap="gray")
    ax.set_title(f"predicted output")
    ax.axis("off")

    # Display the figure.
    plt.show()


def predict_and_save(path_for_saving: str, num_pf_predictions: int = None):
    if num_pf_predictions is None:
        num_pf_predictions = TEST_DATA_SIZE
    next_test = next_data(TEST_PATH, RESIZED_WIDTH, RESIZED_HEIGHT)
    for _ in range(num_pf_predictions):
        pred_input, original_output = next(next_test)
        prediction_output = model.predict(np.array([pred_input]))
        prediction = img_from_output(prediction_output[0], RESIZED_WIDTH, RESIZED_HEIGHT)
        my_input = img_from_input(pred_input,RESIZED_WIDTH,RESIZED_HEIGHT)
        original_output = img_from_output(original_output,RESIZED_WIDTH,RESIZED_HEIGHT)
        #saving:
        join = os.path.join
        volts = pred_input[-3:]
        name = f'{volts[0]}_{volts[1]}_{volts[2]}'
        os.mkdir(join(path_for_saving, name))
        for img_name, img in zip(['prediction.png', 'input.png', 'original_output.png'],
                                 [prediction, my_input, original_output]):
            cv2.imwrite(join(path_for_saving, name, img_name), img)


def show_me_random_unsaved_predictions():
    next_test = next_data(TEST_PATH, RESIZED_WIDTH, RESIZED_HEIGHT)
    while True:
        pred_input, original_output = next(next_test)
        prediction_output = model.predict(np.array([pred_input]))
        prediction = img_from_output(prediction_output[0], RESIZED_WIDTH, RESIZED_HEIGHT)
        my_input = img_from_input(pred_input, RESIZED_WIDTH, RESIZED_HEIGHT)
        original_output = img_from_output(original_output, RESIZED_WIDTH, RESIZED_HEIGHT)
        visualize_prediction(my_input, original_output, prediction)
        que = input('press_enter_for_anther_prediction, type q to break')
        if que == 'q':
            print('breaking...')
            break


def show_me_random_saved_predictions(saved_predictions_path: str):
    predictions_list = os.listdir(saved_predictions_path)
    random.shuffle(predictions_list)
    join = os.path.join
    for pred in predictions_list:
        my_input = cv2.imread(join(saved_predictions_path, pred, 'input.png'), cv2.IMREAD_GRAYSCALE)
        original_output = cv2.imread(join(saved_predictions_path, pred, 'original_output.png'), cv2.IMREAD_GRAYSCALE)
        prediction = cv2.imread(join(saved_predictions_path, pred, 'prediction.png'), cv2.IMREAD_GRAYSCALE)
        visualize_prediction(my_input, original_output, prediction)
        que = input('press_enter_for_anther_prediction, type q to break')
        if que == 'q':
            print('breaking...')
            break
    print('There are no more predictions')




In [ ]:
predict_and_save('H:\hadars_folder\predictions1')

In [ ]:
show_me_random_unsaved_predictions()